In [1]:
import os
from xml.dom import minidom
import pandas as pd
import numpy as np
directory = os.getcwd() 
import nltk
import gzip
import gensim 
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
# Extract text from inside

def preprocess(folder):
    no_of_paras = []
    current_dir = directory + "/"+folder
    base_cases = []
    noticed_cases = []
    paragraphs = []
    entailed_fragments = []
    list_of_files = sorted(os.listdir(current_dir))
    x = 1
    y = len(list_of_files)-x
    for i in range(y):
        print("Processing ",i)
        f = open(current_dir+'/'+list_of_files[i]+"/base_case.txt","r")
        paragraph_filenames = sorted(os.listdir(current_dir+'/'+list_of_files[i]+"/candidates"))
        a = f.read()
        base_cases.append(a)
        f.close()
       # g.close()
        
        arr = []
        no_of_paras.append(len(paragraph_filenames))

        for j in range(len(paragraph_filenames)):
            g = open(current_dir+'/'+list_of_files[i]+"/candidates/"+paragraph_filenames[j],"r")
            paragraphs.append(g.read())
            g.close()
    return base_cases,entailed_fragments,paragraphs,no_of_paras,list_of_files,noticed_cases
    
train_base_cases,train_entailed_fragments, train_paragraphs,train_no_of_paras,train_list_of_files,train_noticed_cases = preprocess('task1_test')
    
#test_base_cases,test_entailed_fragments, test_paragraphs,test_no_of_paras = preprocess('task1_test')

Processing  0
Processing  1
Processing  2
Processing  3
Processing  4
Processing  5
Processing  6
Processing  7
Processing  8
Processing  9
Processing  10
Processing  11
Processing  12
Processing  13
Processing  14
Processing  15
Processing  16
Processing  17
Processing  18
Processing  19
Processing  20
Processing  21
Processing  22
Processing  23
Processing  24
Processing  25
Processing  26
Processing  27
Processing  28
Processing  29
Processing  30
Processing  31
Processing  32
Processing  33
Processing  34
Processing  35
Processing  36
Processing  37
Processing  38
Processing  39
Processing  40
Processing  41
Processing  42
Processing  43
Processing  44
Processing  45
Processing  46
Processing  47
Processing  48
Processing  49
Processing  50
Processing  51
Processing  52
Processing  53
Processing  54
Processing  55
Processing  56
Processing  57
Processing  58
Processing  59
Processing  60


In [22]:
data = train_base_cases + train_paragraphs

In [23]:
def gen_labels(data):
    labels = []
    for i in range(len(train_base_cases)):
        labels.append("base_case_"+train_list_of_files[i])
    for i in range(len(train_base_cases)):
        for j in range(200):
            labels.append("candidate_"+train_list_of_files[i]+"_"+str(j+1))
    return labels
labels = gen_labels(data)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import re
from nltk.corpus import stopwords
from gensim.summarization.bm25 import get_bm25_weights
lemma = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
dict_words = set(nltk.corpus.words.words())
def filt(case_docs):
    filtered_words =[]
    filtered_docs = [ '' for i in range(len(case_docs))]
    lemma_docs = [ '' for i in range(len(case_docs))]
    for i in range(len(case_docs)):
        #print(i)
        case_words = re.split("(?:(?:[^a-zA-Z]+')|(?:'[^a-zA-Z]+))|(?:[^a-zA-Z']+)", case_docs[i])
        filtered_word_list = [word.lower() for word in case_words if (( len(word) >= 3 and word.isalpha() and word.lower() not in stop_words ))  ]
        filtered_words.append(filtered_word_list)
        '''or ( len(word) <= 3 and word.isdigit())'''
        for word in filtered_word_list:
            filtered_docs[i] = filtered_docs[i] + word + " "
            lemma_docs[i] = lemma_docs[i] + lemma.lemmatize(word) + " "
    return lemma_docs


In [20]:
from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec.load("doc2vec_task_1_vec_150_window_10_test_set_minc_2_epoch_50.model")

2019-03-30 13:19:38,604 : INFO : loading Doc2Vec object from doc2vec_task_1_vec_150_window_10_test_set_minc_2_epoch_50.model
2019-03-30 13:19:41,185 : INFO : loading vocabulary recursively from doc2vec_task_1_vec_150_window_10_test_set_minc_2_epoch_50.model.vocabulary.* with mmap=None
2019-03-30 13:19:41,188 : INFO : loading trainables recursively from doc2vec_task_1_vec_150_window_10_test_set_minc_2_epoch_50.model.trainables.* with mmap=None
2019-03-30 13:19:41,191 : INFO : loading wv recursively from doc2vec_task_1_vec_150_window_10_test_set_minc_2_epoch_50.model.wv.* with mmap=None
2019-03-30 13:19:41,193 : INFO : loading docvecs recursively from doc2vec_task_1_vec_150_window_10_test_set_minc_2_epoch_50.model.docvecs.* with mmap=None
2019-03-30 13:19:41,195 : INFO : loaded doc2vec_task_1_vec_150_window_10_test_set_minc_2_epoch_50.model


In [27]:
for i in range(61):
    lst2 = train_base_cases[i:i+1] + train_paragraphs[i*200:(i+1)*200]
    lst3 = filt(lst2)
    arr = []
    for elem in lst3:
        arr.append(elem.split())
    corpus = arr
    result = get_bm25_weights(corpus, n_jobs=-1)
    lst = []
    for j in range(200):
        a = model.docvecs.similarity(labels[i],"candidate_"+train_list_of_files[i]+"_"+str(j+1))
        b = result[0][j+1]
        lst.append((a*b,j+1))
    lst.sort(reverse=True)
    cutoff = ((lst[0][0] + lst[1][0])/2)*0.80
    for j in range(10):
        if lst[j][0] > cutoff:
            print("t1-"+str(i+1),lst[j][1],"IITPdocBM")

t1-1 48 IITPdocBM
t1-2 154 IITPdocBM
t1-3 180 IITPdocBM
t1-3 124 IITPdocBM
t1-4 137 IITPdocBM
t1-4 182 IITPdocBM
t1-4 65 IITPdocBM
t1-5 174 IITPdocBM
t1-5 163 IITPdocBM
t1-5 130 IITPdocBM
t1-5 110 IITPdocBM
t1-5 111 IITPdocBM
t1-6 124 IITPdocBM
t1-6 62 IITPdocBM
t1-6 85 IITPdocBM
t1-6 168 IITPdocBM
t1-6 187 IITPdocBM
t1-6 184 IITPdocBM
t1-6 45 IITPdocBM
t1-6 167 IITPdocBM
t1-6 58 IITPdocBM
t1-6 70 IITPdocBM
t1-7 178 IITPdocBM
t1-7 22 IITPdocBM
t1-8 88 IITPdocBM
t1-8 157 IITPdocBM
t1-8 182 IITPdocBM
t1-9 71 IITPdocBM
t1-10 180 IITPdocBM
t1-10 187 IITPdocBM
t1-10 51 IITPdocBM
t1-10 154 IITPdocBM
t1-11 11 IITPdocBM
t1-11 111 IITPdocBM
t1-11 25 IITPdocBM
t1-11 105 IITPdocBM
t1-11 2 IITPdocBM
t1-11 49 IITPdocBM
t1-11 195 IITPdocBM
t1-12 7 IITPdocBM
t1-12 124 IITPdocBM
t1-12 50 IITPdocBM
t1-12 194 IITPdocBM
t1-12 11 IITPdocBM
t1-12 192 IITPdocBM
t1-12 52 IITPdocBM
t1-13 95 IITPdocBM
t1-14 156 IITPdocBM
t1-14 79 IITPdocBM
t1-15 18 IITPdocBM
t1-16 124 IITPdocBM
t1-16 58 IITPdocBM
t1-17 127 IIT

In [11]:
for j in range(200):
    lst.append((result[0][j+1],j+1))
lst.sort(reverse=True)
cutoff = ((lst[0][0] + lst[1][0])/2)*0.90
for j in range(10):
    if lst[j][0] > cutoff:
        print("t1-"+str(i+1),lst[j][1],"IITPBM25")

In [17]:
result[0][0]

-75020.47405382311

In [ ]:
325,1486,855

In [ ]:
588/2421

In [ ]:
468/1486

In [ ]:
import re
a = re.search("Baban","I am Baban Gain Baban GAin")
print(a)